In [6]:
# The data set used in this example is from http://archive.ics.uci.edu/ml/datasets/Wine+Quality
# P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
# Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.

import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

import logging

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file from the URL
    csv_url = (
        "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    )
    try:
        data = pd.read_csv(csv_url, sep=";")
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e
        )

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    alpha = 1 #float(sys.argv[1]) if len(sys.argv) > 1 else 0.5
    l1_ratio = 1 #float(sys.argv[2]) if len(sys.argv) > 2 else 0.5

    with mlflow.start_run():
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Model registry does not work with file store
        if tracking_url_type_store != "file":

            # Register the model
            # There are other ways to use the Model Registry, which depends on the use case,
            # please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            mlflow.sklearn.log_model(lr, "model", registered_model_name="ElasticnetWineModel")
        else:
            mlflow.sklearn.log_model(lr, "model")

Elasticnet model (alpha=1.000000, l1_ratio=1.000000):
  RMSE: 0.8328280500742669
  MAE: 0.672504249193216
  R2: 0.017246911564584355


In [13]:
!ls -l mlruns/0/fb27c54b21a4452486670b7bd806d477/params

total 8
-rw-r--r-- 1 root root 1 Mar 22 17:52 alpha
-rw-r--r-- 1 root root 1 Mar 22 17:52 l1_ratio


In [17]:
!less mlruns/0/fb27c54b21a4452486670b7bd806d477/metrics/mae

1616435529473 0.672504249193216 0
(END)^C


In [2]:
!pip install mlflow

     |████████████████████████████████| 14.2MB 276kB/s 
     |████████████████████████████████| 81kB 7.2MB/s 
     |████████████████████████████████| 153kB 56.4MB/s 
     |████████████████████████████████| 61kB 6.2MB/s 
     |████████████████████████████████| 163kB 48.1MB/s 
     |████████████████████████████████| 1.1MB 41.2MB/s 
     |████████████████████████████████| 61kB 5.7MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
     |████████████████████████████████| 81kB 6.6MB/s 
  Created wheel for prometheus-flask-exporter: filename=prometheus_flask_exporter-0.18.1-cp37-none-any.whl size=17159 sha256=1fdd2a99d9a9e5cfee310b6f07aa17450500407971842172c038299ecd230de8
  Stored in directory: /root/.cache/pip/wheels/b4/1f/b8/66bd9bc3a9d6c6987ff6c4dfeb6f1fe97b5a0e5ed5849c0437
  Created wheel for databricks-cli: filename=databricks_cli-0.14.3-cp37-none-any.whl size=100557 sha256=b5cd03a60fd7b17fcf8bf7d5ca678569818bb88c073f69847c8528298c5adbf8
  Stored in directory: /root/.cache/pip/